## Ler cada arquivo e grava cada registro na base de dados

In [ ]:
import re
import os
from tqdm import tqdm
from pathlib import Path 
import glob
import pymongo
import datetime

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

logdb = myclient["logdb"]
mycol = logdb["logs"]
pasta = r'/home/ricardo/pucrs/TCC/logs-node1-b2b/**/*.log*'
arqs = glob.glob(pasta, recursive = True)
print("Quantidade de arquivos de log:",len(arqs))

# deleta todas as linhas de log
_ = mycol.delete_many({})
print (_)

registros_log = []

regex_registro = re.compile('^\[(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d+)\](.+)$', re.S)
regex_loop     = re.compile('^\[(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d+)\]')
regex_nmlog    = re.compile('^(.+\.log)(\.)?(.*)$')

def gravaRegistro(linha, path):
    global registros_log
    timestamp = ''
    texto = '' 
    # Recupera data/hora e texto do log
    groups = regex_registro.search(linha)
    if groups:
        timestamp = groups.group(1)
        texto = groups.group(2)

    if (timestamp and texto):
        #
        # Separa o nome do log de sua data/hora
        nomearq1 = os.path.basename(path).split('/')[-1]
        groups   = regex_nmlog.search(nomearq1)
        nomearq  = groups.group(1)
        datahora = groups.group(3)
        #
        dt = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f")        
        registros_log.append({"arq_nome": nomearq, 'arq_dthr': datahora, 'path': path, "timestamp": dt, "texto": texto})
        if (len(registros_log) == 20000):
            mycol.insert_many(registros_log)
            registros_log = []
    return

for arq in tqdm(arqs):
    registro = ''
    if (os.path.isfile(arq)):
        with open(arq,'r',encoding='iso-8859-1') as fh:
            nrlinha = 0
            try:
                for linha in fh:
                    if (len(registro) > 0 and regex_loop.match(linha)):
                        gravaRegistro(registro, arq)
                        registro = linha
                    else:
                        registro += linha
                    #print(repr(linha))
                    nrlinha += 1
            except UnicodeDecodeError as err:
                print(">>> Erro ao ler linhas do arquivo:",arq, "linha:", nrlinha, "Erro:", err)
                exit(0)
                
            if len(registro) > 0:
                gravaRegistro(registro,arq)


if (len(registros_log) > 0):
    mycol.insert_many(registros_log)